#### Imports

In [1]:
import wrangle
import explore
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:20,.2f}'.format

from math import sqrt
from scipy import stats

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression, SelectKBest, RFE 
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import KMeans

#### Import LA Dataframe For Explore

In [2]:
df = wrangle.get_zillow_data(cached=True)
df_la, df_v, df_o = wrangle.clean_zillow_data(df)
X_train, X_validate, X_test, X_train_explore, y_train, y_validate, y_test, X_train_scaled, X_validate_scaled, X_test_scaled = wrangle.split_scale(df_la)

In [3]:
X_train.shape, X_validate.shape, X_test.shape, y_train.shape

((18398, 13), (7886, 13), (6571, 13), (18398, 1))

# Exploration:

**Target = Logerror** 


-A number that represents a ratio that is derived from two prior distributions - the real price distribution of homes and then Zillow's existing model of that distribution

In [4]:
X_train_explore.head()

,logerror,bathroomcnt,calculatedfinishedsquarefeet,latitude,longitude,LA,Orange,Ventura,age,taxrate,acres,structure_dollar_per_sqft,land_dollar_per_sqft,bed_bath_ratio
50712,0.00,3.00,"2,805.00",33.86,-118.36,1,0,0,25.00,0.01,0.13,134.38,104.08,1.33
35937,0.00,2.00,"1,371.00",33.85,-118.16,1,0,0,62.00,0.01,0.13,68.51,66.78,1.00
25473,0.02,3.00,"2,275.00",34.02,-118.06,1,0,0,19.00,0.01,0.14,91.09,26.43,2.00
4302,-0.05,3.00,"1,741.00",34.10,-118.04,1,0,0,36.00,0.01,0.17,100.66,12.79,1.00
7511,-0.13,4.00,"4,127.00",34.17,-118.44,1,0,0,93.00,0.01,0.22,106.74,18.01,1.00


#### Inital Thoughts:

- From my inital investigation on regression project I know that room count has a large affect on taxrate and housing price.  I was unable to create a derived feature last go round so I want to test the affect of this feature now.     

- I want to examine how usefull our created feature of bedbathratio is in predicting logerror in LA County.  I chose LA County because it has the largest number of datapoints.  I want to cluster on bedbathratio, bathroomcnt, and caluculaedfinishedsquarefeet.   

## Cluster One: Room Clusters

#### Step 1. Elbow Plot

In [5]:
# #Reasign for formula to work correctly
# X_train_scaled = X_train_scaled_LA.copy()

cluster_vars = ['bathroomcnt', 'bed_bath_ratio', 'calculatedfinishedsquarefeet']
explore.elbow_plot(X_train_scaled_LA, cluster_vars)

NameError: name 'X_train_scaled_LA' is not defined

#### Takeaway:

- Looks like 3 is the optimal K for this cluster

***

#### Step 2. Create Clusters

#### 2a. Train Cluster

In [ ]:
LA_train_clusters, kmeans = explore.run_kmeans(X_train_LA, X_train_scaled_LA, k=3, cluster_vars=cluster_vars, cluster_col_name = 'room_cluster')

In [ ]:
 LA_train_clusters

In [ ]:
#Visualize distribution of clusters, they do not look even
LA_train_clusters.room_cluster.value_counts()

#### Get Centroids

In [ ]:
centroid_col_names = ['centroid_' + i for i in cluster_vars]
centroid_col_names

LA_centroids = pd.DataFrame(kmeans.cluster_centers_, 
             columns=centroid_col_names).reset_index().rename(columns={'index': 'room_cluster'})

In [ ]:
LA_centroids

#### Append cluster id onto X_train & X_train_scaled, then join with the centroids dataframe.


In [ ]:
# concatenate cluster id on LA_X_Train
X_train_LA_cluster = pd.concat([X_train_LA, LA_train_clusters], axis=1)

In [ ]:
X_train_LA_cluster.head()

In [ ]:
# join on clusterid to get centroids
X_train_LA_cluster_centroid = X_train_LA_cluster.merge(LA_centroids, how='left', on='room_cluster').set_index(X_train_LA_cluster.index)

In [ ]:
X_train_LA_cluster_centroid.head()

#### Clusters and Centroids on Train DF

In [ ]:
# Visualize 

plt.scatter(X_train_LA_cluster_centroid.bathroomcnt, y_train.logerror, c=X_train_LA_cluster_centroid.room_cluster)
plt.show()

#### 2b. Validate Cluster

In [ ]:
LA_validate_clusters, kmeans = explore.run_kmeans(X_validate_LA, X_validate_scaled_LA, k=3, cluster_vars=cluster_vars, cluster_col_name = 'room_clusters')

In [ ]:
LA_validate_clusters

#### 2c. Test Cluster

In [ ]:
LA_test_clusters, kmeans = explore.run_kmeans(X_test_LA, X_test_scaled_LA, k=3, cluster_vars=cluster_vars, cluster_col_name = 'room_clusters')

In [ ]:
LA_test_clusters